# Best areas to go out in a city at night
With the code in this notebook you can find areas with a high density of nightlife venues in a city you are interested in.
Perfect for pub crawls and nightclub hopping!
Data is optained from Foursquare.

In [3]:
# importing necessary packages
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt 
from pylab import rcParams
from mpl_toolkits.basemap import Basemap
%matplotlib inline

# import k-means from clustering stage
#from sklearn.cluster import KMeans

import folium # map rendering library

import geopy
import geocoder 
from geopy.geocoders import Nominatim

from sklearn.cluster import DBSCAN 
from sklearn.datasets.samples_generator import make_blobs 
from sklearn.preprocessing import StandardScaler 
import sklearn.utils

print('Libraries imported.')

Libraries imported.


### Get a location

In [194]:
# let the user decide which city or region he/she wants to explore
print('Please enter a city or address!')
address = input()
print('Perfect, you entered ' + address) 

Please enter a city or address!
Miami Beach
Perfect, you entered Miami Beach


In [195]:
# get geolocation of city 
df_city = pd.DataFrame(columns = ['City','Latitude','Longitude'])

geolocator = Nominatim(user_agent="germany_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of {} are {}, {}.'.format(address , latitude, longitude))

The geograpical coordinates of Miami Beach are 25.7929198, -80.1353006.


### Get nightlife venues nearby with Foursquare API

In [ ]:
# Foursquare API credentials
CLIENT_ID = 'B2GDZFB30ZXR3I0CCC1HPQEOZLRQOI3FBZNTJNUF2NCQK2PN' # your Foursquare ID
CLIENT_SECRET = 'WMRWEZKEXUW4CXJDWMRJZSQUG2D3AU2F5FUHWH0FA3CALXRD' # your Foursquare Secret
ACCESS_TOKEN = '2UCX31JE5X4VIYSV3XPFABPKMFBUW24MDPUP5WKGKW1DWTJI' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version

In [201]:
# choose categories by category ID
category = {'4bf58dd8d48988d11f941735', # Nightclub
            '52e81612bcbc57f1066b7a06', # Irish Pub
            '4bf58dd8d48988d1d5941735', # Hotel Bar
            '4bf58dd8d48988d121941735', # Lounge
            '4bf58dd8d48988d11e941735', # Cocktail Bar
            '4bf58dd8d48988d116941735', # Bar
            '4bf58dd8d48988d11b941735'} # Pub

In [202]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# creat the url to search for venues to go out to at night
df_night = pd.DataFrame()
for id in category:
    LIMIT = 1000
    radius = 10000
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        latitude, 
        longitude,
        ACCESS_TOKEN, 
        VERSION, 
        id, 
        radius, 
        LIMIT)
    #print(url)

    

    #get results of Foursquare request
    results = requests.get(url).json()
    
    venues = results['response']['venues'] # assign relevant part of JSON to venues
    dataframe = json_normalize(venues) # tranform venues into a dataframe

    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]

    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1) # filter the category for each row
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns] # clean column names by keeping only last term
    
    df_night = df_night.append(dataframe_filtered, ignore_index=True)

df_night

C:\Users\jenni\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,address,categories,cc,city,country,crossStreet,distance,formattedAddress,id,labeledLatLngs,lat,lng,name,neighborhood,postalCode,state
0,1115-1117 16th St,Brewery,US,Miami Beach,United States,NaN,665,"[1115-1117 16th St, Miami Beach, FL 33139, Uni...",49e111a9f964a5208e611fe3,"[{'label': 'display', 'lat': 25.78904793643717...",25.789048,-80.140365,Abbey Brewing Company,NaN,33139,FL
1,1551-1599 Florida 907,Pub,US,Miami Beach,United States,NaN,1018,"[1551-1599 Florida 907, Miami Beach, FL 33139,...",4c551736fd2ea5937ae1222b,"[{'label': 'display', 'lat': 25.78416607281431...",25.784166,-80.132333,Playwright Sports Bar,NaN,33139,FL
2,Brickell,Pub,US,Miami,United States,NaN,7558,"[Brickell, Miami, FL, United States]",4e3658d1a8092599ee0e3334,"[{'label': 'display', 'lat': 25.753912, 'lng':...",25.753912,-80.197015,Calexico's Pub,NaN,NaN,FL
3,3120 Collins Ave,Pub,US,Miami Beach,United States,NaN,1911,"[3120 Collins Ave, Miami Beach, FL 33140, Unit...",5bde729dba57b4002cafc723,"[{'label': 'display', 'lat': 25.807203, 'lng':...",25.807203,-80.124711,The Driftway,NaN,33140,FL
4,NaN,Pub,US,Miami,United States,NaN,6399,"[Miami, FL, United States]",554429c7498e6f518b7b2dbe,"[{'label': 'display', 'lat': 25.80325031542983...",25.803250,-80.198112,bxldr,NaN,NaN,FL
5,1109 Brickell Ave,Pub,US,Miami,United States,at JW Marriott Hotel Miami,6592,[1109 Brickell Ave (at JW Marriott Hotel Miami...,4bea076ea9900f47cf201640,"[{'label': 'display', 'lat': 25.76244191616899...",25.762442,-80.191688,Drake's Bar,NaN,33131,FL
6,4594 36th street,Pub,US,Miami,United States,NaN,13060,"[4594 36th street, Miami, FL, United States]",4b242de0f964a520126324e3,"[{'label': 'display', 'lat': 25.80819667705229...",25.808197,-80.264509,Toms Bar,NaN,NaN,FL
7,900 S Miami Avenue 161,Food Truck,US,Miami,United States,NaN,6789,"[900 S Miami Avenue 161, Miami, FL 33130, Unit...",5e3ae8ad346b7c0008c863f5,"[{'label': 'display', 'lat': 25.76242556028679...",25.762426,-80.193955,Moxie's,NaN,33130,FL
8,1111 SW 1st Ave #107,Bar,US,Miami,United States,NaN,6810,"[1111 SW 1st Ave #107, Miami, FL 33130, United...",5a6280e22d2fd96f6d9dcd8b,"[{'label': 'display', 'lat': 25.7631636, 'lng'...",25.763164,-80.194670,Bitter Truth,NaN,33130,FL
9,938 Lincoln Rd,Pub,US,Miami Beach,United States,NaN,444,"[938 Lincoln Rd, Miami Beach, FL 33139, United...",50b11970e4b056d2aba97a5d,"[{'label': 'display', 'lat': 25.79022026062011...",25.790220,-80.138573,Buddha,NaN,33139,FL


### Density based clustering of venues

In [203]:
sklearn.utils.check_random_state(1000)
Clus_dataSet = df_night[['lat','lng']]
Clus_dataSet = np.nan_to_num(Clus_dataSet)
Clus_dataSet = StandardScaler().fit_transform(Clus_dataSet)

# Compute DBSCAN
db = DBSCAN(eps=0.17, min_samples=3).fit(Clus_dataSet)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
df_night["Clus_Db"]=labels

realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
clusterNum = len(set(labels)) 


# A sample of clusters
df_night[["name","categories","lat","lng", "Clus_Db"]].head(5)

,name,categories,lat,lng,Clus_Db
0,Abbey Brewing Company,Brewery,25.789048,-80.140365,0
1,Playwright Sports Bar,Pub,25.784166,-80.132333,1
2,Calexico's Pub,Pub,25.753912,-80.197015,-1
3,The Driftway,Pub,25.807203,-80.124711,1
4,bxldr,Pub,25.803250,-80.198112,2


### Visualization

In [199]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the city center (or address)

col = ['red','blue','green','purple','orange','darkred','lightred','darkblue','darkpurple','lightgreen','darkgreen','cadetblue','lightblue','#ff7f50','#81d8d0','#ccff00','#7fe5f0','#ff0000','#ff80ed','#407294','#cbcba9','#468499','#f6546a','#afeeee','#008000','#660066','#cbbeb5','#ff4040','#bada55','#f5f5dc','beige']
# add the venues as blue circle markers
for lat, lng, category, name, cluster in zip(df_night.lat, df_night.lng, df_night.categories, df_night.name, df_night.Clus_Db):
    label = folium.Popup(str(name) + ' -\n' + str(category), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=col[cluster],
        popup= label,
        fill = True,
        fill_color=col[cluster],
        fill_opacity=0.8
    ).add_to(venues_map)
    
# add a red circle marker to represent the center
folium.Marker(
    [latitude, longitude],
    popup='Center',
).add_to(venues_map)

# display map
venues_map

The Marker shows the location you typed in. The dots present venues where you can go out at night (bar, nightclub, pub,...). The colored clusters show areas with a high density of these venues and thus, suggest areas where you can visit many clubs, bars, pubs etc. Without walking to far. Perfect for a pub crawl or nightclub hopping! Venues that do not belong to a cluster are however shown as less visible beige dots.
Note, that there is a limit of 50 venues per request with the free account. If there wasn't a limit of 50 venues the clusters would be more informative.